In [16]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [17]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where r_comments like 'Verified%' and product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

def retrieve_all_reviews():
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where r_comments like 'Verified%' ")
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

In [18]:
#df = retrieve_reviews('Samsung Electronics UN65MU6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')
df = retrieve_all_reviews()
columns = ['id','product','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']
df.columns = columns
df.head(2)

,id,product,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [19]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new['reviews'] = df['r_title'] + ' ' + df['r_text']
df_new = df_new[['rating','product','reviews']]
df_new.head(3)

,rating,product,reviews
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,"TV Great, TruMotion Bad! Now that I found the ..."
1,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,Hint: Turn OFF Smart Picture Mode Hint: Turn o...
2,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,43UJ6300 Needed an upgrade for my 7yr old HDTV...


In [5]:
from src.LdaReviewAnalyzer import LdaReviewAnalyzer

In [13]:
lda = LdaReviewAnalyzer(df_new)
lda.fit_transform(num_topics=2, rating=[1])
lda.print_topics()

Topic 0:
[('great', 254.29354568923878), ('picture', 253.11311981366373), ('samsung', 203.83808797237322), ('quality', 182.45487275799618), ('love', 171.563016182366), ('remote', 148.068290944625), ('price', 120.59535164746147), ('smart', 105.04065780060523), ('good', 99.60263897617126), ('use', 89.4555797053181)]
Topic 1:
[('great', 429.2885084134207), ('stars', 399.18039260168547), ('picture', 283.41964833909833), ('set', 131.35618557508522), ('easy', 119.20706536723523), ('good', 107.26057922498289), ('excellent', 104.62356882155433), ('nice', 86.24003940903329), ('sound', 85.81805434378728), ('price', 70.8374360058537)]


In [14]:
import warnings
warnings.filterwarnings('ignore')
widget = lda.display_topic_model()
widget

In [36]:
from src.w2v_review_analyzer import W2VReviewAnalyzer

In [37]:
w2v = W2VReviewAnalyzer(df_new)


In [46]:
w2v.fit()

In [56]:
w2v.model.wv.most_similar('tv')

[('picture_quality', 0.9997904300689697),
 ('easy_set', 0.9997736215591431),
 ('fantastic', 0.9997535943984985),
 ('amazing', 0.9997393488883972),
 ('quality', 0.999700665473938),
 ('nice', 0.9996711611747742),
 ('feature', 0.9996548295021057),
 ('tv.', 0.9996519684791565),
 ('sound', 0.9996477365493774),
 ('clear', 0.9996407628059387)]